In [1]:
import math

import torch

from script.NeuralNets.Networks import SequentialNN, ICNN, ICNNApproxMax, ICNNLogical
from script.settings import device, data_type
import script.DHOV.MultiDHOV as multidhov
from script.Verification.Verifier import SingleNeuronVerifier, MILPVerifier, DHOVVerifier
import gurobipy as grp
from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms import Compose, ToTensor, Normalize
from script.NeuralNets.ICNNFactory import ICNNFactory

In [2]:
def add_max_constr(model, neuron_name):
    neuron_var = model.getVarByName(neuron_name)
    model.setObjective(neuron_var, grp.GRB.MAXIMIZE)

def add_min_constr(model, neuron):
    neuron_var = model.getVarByName(neuron)
    model.setObjective(neuron_var, grp.GRB.MINIMIZE)

In [3]:
def optimize_model(model, neuron_name):
    model.update()
    model.optimize()
    if model.Status == grp.GRB.OPTIMAL:
        print("opt value: {}".format(model.getVarByName(neuron_name).getAttr("x")))

In [4]:
def icnn_model(icnn, nn, input_x, eps, layer_index, from_neuron, to_neuron, print_log=False):
    m = grp.Model()
    if not print_log:
        m.Params.LogToConsole = 0

    input_flattened = torch.flatten(input_x)
    bounds_affine_out, bounds_layer_out = nn.calculate_box_bounds(
        [input_flattened.add(-eps), input_flattened.add(eps)])

    parameter_list = list(nn.parameters())

    input_size = len(parameter_list[2*(layer_index-1)])
    lb = bounds_layer_out[layer_index-1][0].detach().cpu().numpy()
    ub = bounds_layer_out[layer_index-1][1].detach().cpu().numpy()
    in_var = m.addMVar(input_size, lb=-float("inf"), ub=float("inf"), name="icnn_var")

    low = bounds_layer_out[layer_index - 1][0][from_neuron: to_neuron]
    up = bounds_layer_out[layer_index - 1][1][from_neuron: to_neuron]
    low = torch.zeros_like(low, dtype=data_type).to(device) - 1000
    up = torch.zeros_like(low, dtype=data_type).to(device) + 1000
    constraint_bounds_affine_out, constraint_bounds_layer_out = icnn.calculate_box_bounds([low, up])
    icnn.add_max_output_constraints(m, in_var[from_neuron: to_neuron], constraint_bounds_affine_out, constraint_bounds_layer_out)

    return m

In [5]:
"""W1 = [1. 1.; 1. -1.]
    b1 = [0., 0.]
    W2 = [1. 1.; 1. -1.]
    b2 = [-0.5, 0.]
    W3 = [-1. 1.; 1. 1.]
    b3 = [3., 0.] """

"""nn = SequentialNN([2, 2, 2, 2])

with torch.no_grad():
    parameter_list = list(nn.parameters())
    parameter_list[0].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[1].data = torch.tensor([0, 0], dtype=data_type).to(device)
    parameter_list[2].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[3].data = torch.tensor([-0.5, 0], dtype=data_type).to(device)
    parameter_list[4].data = torch.tensor([[-1, 1], [1, 1]], dtype=data_type).to(device)
    parameter_list[5].data = torch.tensor([3, 0], dtype=data_type).to(device)

test_image = torch.tensor([[0, 0]], dtype=data_type).to(device)"""

"""transform = Compose([ToTensor(),
                         Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
                        )

training_data = CIFAR10(root="../../cifar", train=True, download=True, transform=transform)
images, labels = training_data.__getitem__(0)
test_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(
    torch.tensor(labels), 0).to(dtype=data_type).to(device)

nn = SequentialNN([32 * 32 * 3, 1024, 512, 10])
nn.load_state_dict(torch.load("../../cifar_fc.pth", map_location=torch.device(device)), strict=False)
"""

transform = Compose([ToTensor(),
                         Normalize(0.5, 0.5)]
                        )

training_data = MNIST(root="../../mnist",
                      train=True,
                      download=True,
                      transform=transform)
images, labels = training_data.__getitem__(0)
test_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(
    torch.tensor(labels), 0).to(dtype=data_type).to(device)

nn = SequentialNN([28*28*1, 100, 30, 10])
nn.load_state_dict(torch.load("../../mnist_fc.pth", map_location=torch.device('cpu')), strict=False)

parameter_list = list(nn.parameters())


"""nn = SequentialNN([50, 50, 50, 7])
test_image = torch.zeros((1, 50), dtype=data_type).to(device)
parameter_list = list(nn.parameters())"""

'nn = SequentialNN([50, 50, 50, 7])\ntest_image = torch.zeros((1, 50), dtype=data_type).to(device)\nparameter_list = list(nn.parameters())'

In [6]:
print(nn(test_image))
print(test_label)

tensor([[  0.5661,   1.4250,   0.4501,   9.2822, -12.7331,  10.6958,  -3.4634,
           2.1465,  -3.7246,  -2.3831]], dtype=torch.float64,
       grad_fn=<AddmmBackward0>)
tensor([5.], dtype=torch.float64)


In [7]:
eps = 0.01

Test for DHOV

In [8]:
group_size = 2
icnn_factory = ICNNFactory("logical", [10, 10, 1], force_positive_init=False, with_two_layers=False,
                               init_scaling=10, init_all_with_zeros=False)
#icnn_factory = ICNNFactory("standard", [5, 5, 1])

dhov_verifier = multidhov.MultiDHOV()
dhov_verifier.start_verification(nn, test_image, icnn_factory, group_size, eps=eps, icnn_epochs=100,
                                 icnn_batch_size=1000, sample_count=1000, sample_new=True, use_over_approximation=True, break_after=None,
                                 sample_over_input_space=False, sample_over_output_space=True, use_icnn_bounds=True,
                                 use_fixed_neurons=True, sampling_method="per_group_sampling",
                                 force_inclusion_steps=0, preemptive_stop=True, even_gradient_training=False,
                                 keep_ambient_space=True, data_grad_descent_steps=0, opt_steps_gd=0,
                                 train_outer=False, print_training_loss=False, print_new_bounds=True,
                                 should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="included")


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-12


C:\Users\Ufuk\Documents\Programming\ICNN_verification\script\DHOV\MultiDHOV.py:76: UserWarning: keep_ambient_space is True and sampling method is per_group_sampling. Keeping previous samples is not supported when using per group sampling
  warnings.warn("keep_ambient_space is True and sampling method is per_group_sampling. "



approximation of layer: 0
    number of fixed neurons for current layer: 92
    layer progress, group 1 of 4 
        time for sampling for one group: 0.008031606674194336


C:\Users\Ufuk\Documents\Programming\ICNN_verification\script\Optimizer\sdlbfgs.py:83: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ..\torch\csrc\utils\python_arg_parser.cpp:1420.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))


        time for training: 4.011852264404297
        actual verification time 0.022167205810546875
        time for verification: 0.1208944320678711
    layer progress, group 2 of 4 
        time for sampling for one group: 0.01005244255065918
        time for training: 3.7841906547546387
        actual verification time 0.030219316482543945
        time for verification: 0.12119388580322266
    layer progress, group 3 of 4 
        time for sampling for one group: 0.008075714111328125
        time for training: 3.5813701152801514
        actual verification time 0.022216320037841797
        time for verification: 0.14115452766418457
    layer progress, group 4 of 4 
        time for sampling for one group: 0.010082483291625977
        time for training: 3.440850019454956
        actual verification time 0.020176410675048828
        time for verification: 0.1289839744567871
    time for regrouping method: 0.0

approximation of layer: 1
        0, lower: new -0.0794514874905275, old -1.

In [84]:
layer_index = 0
neuron_index = 23
neuron_name = "relu_var{}[{}]".format(2*layer_index, neuron_index)
# neuron_name = "last_affine_var[{}]".format(neuron_index)

In [74]:
print(dhov_verifier.all_group_indices)

[[[1, 23], [25, 51], [60, 69], [86, 96]], [[0]]]


In [75]:
print(dhov_verifier.bounds_affine_out[layer_index][0])

tensor([ 5.0935e-03,  7.5383e-01, -5.7528e-01,  8.4485e+00, -1.3539e+01,
         9.9033e+00, -4.1025e+00,  1.4745e+00, -4.4551e+00, -3.1310e+00],
       dtype=torch.float64, grad_fn=<CopySlices>)


In [76]:
print(dhov_verifier.bounds_affine_out[layer_index][1])

tensor([  1.1184,   2.0946,   1.4067,  10.0374, -11.8526,  11.4839,  -2.8244,
          2.8412,  -2.9536,  -1.6294], dtype=torch.float64,
       grad_fn=<CopySlices>)


In [77]:
input_flattened = torch.flatten(test_image)
simple_bounds_affine_out, simple_bounds_layer_out = nn.calculate_box_bounds([input_flattened.add(-eps), input_flattened.add(eps)])
print(simple_bounds_affine_out[layer_index][0])
print(simple_bounds_affine_out[layer_index][1])

tensor([ -5.2223,  -4.9188,  -6.6589,   2.6449, -18.6182,   3.6405,  -9.5669,
         -4.3524, -10.9687,  -9.1357], dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor([ 6.4550,  7.2957,  7.8522, 15.6069, -5.7168, 17.2791,  2.3780,  8.4534,
         3.7083,  4.3367], dtype=torch.float64, grad_fn=<AddBackward0>)


In [78]:
icnn_neuron_name = "output_layer_[{}]_[{}]".format(layer_index, neuron_index)

In [79]:
dhov_copy = dhov_verifier.nn_encoding_model.copy()
add_min_constr(dhov_copy, icnn_neuron_name)
dhov_copy.update()
optimize_model(dhov_copy, icnn_neuron_name)

opt value: 0.00509347543601657


In [80]:
dhov_copy = dhov_verifier.nn_encoding_model.copy()
dhov_copy.Params.LogToConsole = 0
add_max_constr(dhov_copy, icnn_neuron_name)
optimize_model(dhov_copy, icnn_neuron_name)

opt value: 1.1183649847797343


In [87]:
nn_input_as_lp = []
for i in range(parameter_list[0].shape[1]):
    nn_input_as_lp.append(dhov_copy.getVarByName("output_layer_[-1]_[{}]".format(i)).getAttr("x"))
nn_input_as_lp = torch.tensor(nn_input_as_lp, dtype=data_type).to(device)

neuron_value_in_lp = []
for layer_index in range(0, len(parameter_list) // 2):
    current_layer_as_lp = []
    for i in range(parameter_list[layer_index * 2].shape[0]):
        current_layer_as_lp.append(dhov_copy.getVarByName("output_layer_[{}]_[{}]".format(layer_index, i)).getAttr("x"))
    neuron_value_in_lp.append(torch.tensor(current_layer_as_lp, dtype=data_type).to(device))

KeyboardInterrupt: 

Test for SNV

In [81]:
snv_verifier = SingleNeuronVerifier(nn, test_image, eps, print_log=False)
snv_verifier.generate_constraints_for_net()
snv_model = snv_verifier.model
snv_model.update()

================ layer 0 ===============
================ layer 1 ===============
        0, lower: new -0.01432710402877753, old -1.0272694263485382
        0, upper: new 0.5730239543289277, old 1.590265447932639
        1, lower: new 5.6693464021433, old 4.891074441089023
        1, upper: new 6.142891226457357, old 6.908179377487894
        2, lower: new 1.2133011004497163, old 0.4758416777957901
        2, upper: new 1.6912240704039898, old 2.45705423241999
        3, lower: new -3.5531681529549597, old -4.154560614460519
        3, upper: new -3.2485968330353114, old -2.6379028626565093
        4, lower: new 1.6869863275604329, old 0.6517725440588809
        4, upper: new 2.2962018490677614, old 3.3335162230102693
        5, lower: new -1.2584008138560365, old -2.143489793168459
        5, upper: new -0.7834514746218495, old 0.0854374951093364
        6, lower: new 0.5966294908309514, old -0.409115897681394
        6, upper: new 1.1930418268841008, old 2.2013279327856976
        7

In [85]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_min_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 0.07810725344285707


In [86]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_max_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 1.0446332572036505


In [34]:
all_vars = snv_copy.getVars()
for var in all_vars:
    print("{} : {}".format(var.getAttr("varname"), var.getAttr("x")))

in_var[0] : -1.01
in_var[1] : -1.01
in_var[2] : -0.99
in_var[3] : -0.99
in_var[4] : -1.01
in_var[5] : -1.01
in_var[6] : -0.99
in_var[7] : -0.99
in_var[8] : -0.99
in_var[9] : -0.99
in_var[10] : -0.99
in_var[11] : -0.99
in_var[12] : -1.01
in_var[13] : -0.99
in_var[14] : -0.99
in_var[15] : -1.01
in_var[16] : -0.99
in_var[17] : -0.99
in_var[18] : -1.01
in_var[19] : -0.99
in_var[20] : -0.99
in_var[21] : -1.01
in_var[22] : -1.01
in_var[23] : -1.01
in_var[24] : -1.01
in_var[25] : -1.01
in_var[26] : -0.99
in_var[27] : -1.01
in_var[28] : -1.01
in_var[29] : -0.99
in_var[30] : -1.01
in_var[31] : -1.01
in_var[32] : -1.01
in_var[33] : -0.99
in_var[34] : -1.01
in_var[35] : -0.99
in_var[36] : -1.01
in_var[37] : -1.01
in_var[38] : -1.01
in_var[39] : -1.01
in_var[40] : -1.01
in_var[41] : -0.99
in_var[42] : -0.99
in_var[43] : -1.01
in_var[44] : -1.01
in_var[45] : -0.99
in_var[46] : -1.01
in_var[47] : -0.99
in_var[48] : -1.01
in_var[49] : -0.99
in_var[50] : -1.01
in_var[51] : -1.01
in_var[52] : -0.99
in_

Test for MILP

In [16]:
milp_verifier = MILPVerifier(nn, test_image, eps, print_log=False)
milp_verifier.generate_constraints_for_net(until_layer_neuron=[layer_index, neuron_index])
milp_model = milp_verifier.model
milp_model.update()

"""all_var = milp_model.getVars()
for var in all_var:
    print(var)"""


'all_var = milp_model.getVars()\nfor var in all_var:\n    print(var)'

In [17]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_min_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 0.0


In [18]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_max_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 0.5622137652362753
